In [2]:
import os
import json
import pandas as pd
import numpy as np
import miditoolkit
from miditoolkit import MidiFile

from miditok import REMI, TokenizerConfig
from symusic import Score
from pathlib import Path
from symusic import Score

In [3]:
df = pd.read_csv("midi_df.csv")
midi_files = df["midi_filename"].apply(lambda x: "maestro-v3.0.0/" + x).to_list()
midi_files[:10]

['maestro-v3.0.0/2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.midi',
 'maestro-v3.0.0/2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MID--AUDIO_03_R2_2008_wav--2.midi',
 'maestro-v3.0.0/2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-split_07-07-17_Piano-e_3-02_wav--3.midi',
 'maestro-v3.0.0/2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MID--AUDIO_21_R1_2004_01_Track01_wav.midi',
 'maestro-v3.0.0/2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MID--AUDIO_17_R1_2006_04_Track04_wav.midi',
 'maestro-v3.0.0/2009/MIDI-Unprocessed_07_R1_2009_04-05_ORIG_MID--AUDIO_07_R1_2009_07_R1_2009_04_WAV.midi',
 'maestro-v3.0.0/2009/MIDI-Unprocessed_11_R1_2009_06-09_ORIG_MID--AUDIO_11_R1_2009_11_R1_2009_07_WAV.midi',
 'maestro-v3.0.0/2013/ORIG-MIDI_03_7_8_13_Group__MID--AUDIO_19_R2_2013_wav--3.midi',
 'maestro-v3.0.0/2009/MIDI-Unprocessed_02_R1_2009_03-06_ORIG_MID--AUDIO_02_R1_2009_02_R1_2009_04_WAV.midi',
 'maestro-v3.0.0/2009/MIDI-Unprocessed_02_R1_2009_03-06_ORIG_MID--AUDIO_02_R1_2009_02_R1_2009_0

In [4]:
config = TokenizerConfig(num_velocities=1, use_chords=False, use_programs=False)
tokenizer = REMI(config)
tokenizer.train(vocab_size=len(midi_files), files_paths=midi_files)

In [ ]:
token_seqs = []

for abs_path in midi_files:
    score = Score(abs_path)
    tokens = tokenizer(score)
    token_ids = tokens[0].ids if isinstance(tokens, list) else tokens.ids
    token_seqs.append(token_ids)

df["token_sequence"] = token_seqs

In [8]:
df.head()

,Unnamed: 0,canonical_composer,canonical_title,split,year,midi_filename,duration,filename,duration_ticks,tempo_bpm,num_notes,avg_pitch,pitch_range,unique_pitches,token_sequence
0,0,Alban Berg,Sonata Op. 1,train,2018,2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R...,698.661160,MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_201...,540659,120.0,4197,65.149392,75,72,"[803, 51, 316, 187, 314, 307, 62, 438, 55, 505..."
1,1,Alban Berg,Sonata Op. 1,train,2008,2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MI...,759.518471,MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MID--AU...,587304,120.0,4206,65.247741,75,74,"[803, 51, 326, 185, 572, 282, 62, 478, 45, 511..."
2,2,Alban Berg,Sonata Op. 1,train,2017,2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-...,464.649433,MIDI-Unprocessed_066_PIANO066_MID--AUDIO-split...,447225,120.0,3326,65.996091,75,72,"[371, 51, 326, 174, 528, 178, 62, 478, 45, 480..."
3,3,Alexander Scriabin,"24 Preludes Op. 11, No. 13-24",train,2004,2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MI...,872.640588,MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MID--AU...,837875,120.0,6316,60.581856,81,78,"[256, 175, 54, 511, 50, 569, 176, 42, 495, 186..."
4,4,Alexander Scriabin,"3 Etudes, Op. 65",validation,2006,2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MI...,397.857508,MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MID--AU...,306880,120.0,4019,68.433192,83,82,"[966, 356, 929, 334, 352, 179, 292, 182, 299, ..."


In [9]:
new_tokens = df["token_sequence"]

In [10]:
old_tokens = pd.read_csv("midi_df.csv")["token_sequence"]

In [11]:
new_tokens.apply(len)

0        7263
1        7671
2        5765
3       10172
4        5864
        ...  
1271     1732
1272     1751
1273    13622
1274    13605
1275     8607
Name: token_sequence, Length: 1276, dtype: int64

In [12]:
old_tokens.apply(len)

0        72006
1        71227
2        57047
3       108195
4        67634
         ...  
1271     15800
1272     21759
1273    135367
1274    135503
1275     97200
Name: token_sequence, Length: 1276, dtype: int64

In [15]:
df = df[df.columns[1:]]

In [ ]:
# df.to_csv("midi_df_v2.csv", index=False)